In [2]:
import pandas as pd
import numpy as np
from IPython.display import display, display_pretty, Javascript, HTML
from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plt

# Show charts in notebook
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# adding an empty line to test

In [4]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [5]:
# Load 2014 and 2015 survey household data
hh14 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-households.xlsx', sheet='Data')
hh15 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2015\Data\Household\1_PSRC2015_HH_2015-07-27_V1.2.xlsx', sheet='Data')

In [6]:
# Load the combined 2014-2015 dataset to get expansion weights
hh14_15 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Combined\1_Combined_HH_2015-11-17.xlsx', sheet='Data')

In [7]:
# Select only the 2015 household records
hh14_15 =hh14_15[hh14_15['datasource'] == 'hts2015']
len(hh14_15)

2428

In [20]:
hh15 = pd.merge(hh15,hh14_15[['hhid','expwt_h1415']],on='hhid')

In [22]:
hh15panel = hh15[hh15['sampletype'] == 2]

In [59]:
expwt15 = 'expwt_h1415_x'
expwt15panel = 'expwt_h1415_x'

hh15.year = 'cross-section'
hh15panel.year = 'panel'

In [47]:
hh15.columns

Index([u'chk', u'done', u'notes', u'hhid', u'sampletype', u'tacoma_addon', u'traveldate', u'dayofweek', u'hhnumtrips', u'vehicle_count', u'hhsize', u'numadults', u'numchildren', u'numworkers', u'lifecycle', u'hh_income_detailed', u'hh_income_followup', u'hh_income_broad', u'sample_segname', u'msg_haddress', u'msg_hlat', u'msg_hlng', u'rep_haddress', u'rep_hlat', u'rep_hlng', u'h_address', u'h_lat', u'h_lng', u'address_use_flag', u'panel_moveinspect', u'h_segname', u'h_county_name', u'h_taz2010', u'h_district_name', u'h_rgc_name', u'h_school_district_name', u'h_uv_name', u'h_uv_group', u'h_tract', u'h_block', u'h_bg', u'h_puma12', u'res_months', u'res_dur', u'rent_own', u'res_type', u'res_factors_hhchange', u'res_factors_afford', u'res_factors_school', u'res_factors_walk', u'res_factors_space', u'res_factors_closefam', u'res_factors_transit', u'res_factors_hwy', u'res_factors_30min', u'prev_rent_own', u'prev_res_type', u'prev_home_wa', u'prev_home_loc_address', u'prev_home_lat', u'prev_

In [48]:
df1 = hh15
df2 = hh15panel

In [49]:
# Add labels for worker type
labels = {
1: " <10k",
2: " 10k-25k",
3: " 25k-35k",
4: " 35k-50k",
5: " 50k-75k",
6: " 75k-100k",
7: "100k-150k",
8: "150k-200k",
9: "200k-250k",
10: ">250k",
98: "Prefer not to answer"}


df1['hh_income_detailed'] = ([labels[x] for x in df1['hh_income_detailed']])
df2['hh_income_detailed'] = ([labels[x] for x in df2['hh_income_detailed']])

KeyError: ' 50k-75k'

In [50]:
labels = {
    0:'no response',
1:'Single-family house (detached house)',
2:'Townhouse (attached house)',
3:'Multi-family house (3 or fewer apartments)',
4:'Building with 3 or fewer apartments/condos',
5:'Building with 4 or more apartments/condos',
6:'Mobile home/trailer',
7:'Dorm or institutional housing',
8:'Other (including boat, RV, van, etc.)',
}

df1['res_type'] = ([labels[x] for x in df1['res_type']])
df2['res_type'] = ([labels[x] for x in df2['res_type']])

df1['prev_res_type'].fillna(0,inplace=True)
df2['prev_res_type'].fillna(0,inplace=True)

df1['prev_res_type'] = ([labels[x] for x in df1['prev_res_type']])
df2['prev_res_type'] = ([labels[x] for x in df2['prev_res_type']])

KeyError: 'Building with 4 or more apartments/condos'

In [51]:
# Replace RGC name on 2015 with matching 2014 name for Seattle RGCs
labels = {
    'no_rgc': 'Outside RGC',
    'Seattle CBD': 'Seattle Downtown',
    'First Hill/Capitol Hill': 'Seattle First Hill/Capitol Hill',
    'South Lake Union': 'South Lake Union',
    'Uptown': 'Seattle Uptown',
    'Seattle University Community': 'Seattle University Community',
    'Northgate': 'Seattle Northgate'
}

hh15['h_rgc_name'].fillna('no_rgc',inplace=True)
hh14['h_rgc_name'].fillna('no_rgc',inplace=True)

In [52]:
hh15['h_rgc_name'].replace(to_replace=labels.keys(), value=labels.values(), inplace=True, limit=None, regex=False, method='pad', axis=None)
hh14['h_rgc_name'].replace(to_replace=labels.keys(), value=labels.values(), inplace=True, limit=None, regex=False, method='pad', axis=None)

In [53]:
# Add

In [54]:
def df_sample(df1, df2, field, agg1, agg2, report_avg=True, print_df=False):
    '''Compare samples from two surveys in the same dataframe'''
    df = pd.DataFrame()
    df[df1.year + ' Samples'] = df1.groupby(field).count()[agg1]
    df[df2.year + ' Samples'] = df2.groupby(field).count()[agg2]
    df[" "*5] = ""    # empty col to separate samples from shares
    df[df1.year + ' Shares'] = (df1.groupby(field).count()[agg1]/df1[agg1].count()).round(2)
    df[df2.year + ' Shares'] = (df2.groupby(field).count()[agg2]/df2[agg2].count()).round(2)

    if print_df:
        print df  

        print "---------" * 10

        if report_avg:
            print "Sample average " + df1.year + ": " + str(np.round(df1[field].mean(),2))
            print "Sample average " + df2.year + ": " + str(np.round(df2[field].mean(),2))
            
    else:
        return df

In [55]:
def df_weighted(df1, df2, field, agg1, agg2):
    '''Compare weighted results from two data sources'''
    df = pd.DataFrame()
    df[df1.year] = df1.groupby(field).sum()[agg1]
    df[df2.year] = df2.groupby(field).sum()[agg2]
    df[" "*5] = ""    # empty col to separate samples from shares
    df[df1.year + ' Shares'] = (df1.groupby(field).sum()[agg1]/df1[agg1].sum()).round(2)
    df[df2.year + ' Shares'] = (df2.groupby(field).sum()[agg2]/df2[agg2].sum()).round(2)
    
    return df

In [56]:
def df_mean(df, fieldname):
    '''Calculate mean for categorical dataframes'''
    mean = {}
    mean[df1.year] = (df[df1.year]*df.index).sum()/(df[df1.year].sum())
    mean[df2.year] = (df[df2.year]*df.index).sum()/(df[df2.year].sum())
    print 'Mean - ' + fieldname
    print df1.year + ': ' + str(mean[df1.year])
    print df2.year + ': ' + str(mean[df2.year])

---

** Adults per Household **

In [60]:
fieldname = 'Adults per Household'
df = df_weighted(hh15, hh15panel, 'numadults', expwt15, expwt15panel)
df.fillna(0,inplace=True)
df

,cross-section,panel,,cross-section Shares,panel Shares
numadults,,,,,
1,125810.749990,125810.749990,,0.26,0.26
2,270832.675210,270832.675210,,0.55,0.55
3,56938.384661,56938.384661,,0.12,0.12
4,30610.239128,30610.239128,,0.06,0.06
5,3833.055738,3833.055738,,0.01,0.01


In [61]:
df_mean(df, fieldname)

Mean - Adults per Household
cross-section: 2.00788336523
panel: 2.00788336523


In [62]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname, xlim=(0,4))

---

** Household Size **

In [20]:
fieldname = 'Household Size'
df = df_weighted(hh14, hh15, 'hhsize', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
hhsize,,,,,
1,422516.9981,437426.909433,,0.29,0.29
2,492845.9979,516130.841258,,0.34,0.34
3,231467.9990,242116.040001,,0.16,0.16
4,194682.9999,202476.688715,,0.13,0.13
5,81309.2959,77221.255288,,0.06,0.05
6,28451.7610,26045.524307,,0.02,0.02
7,9152.5333,21658.006983,,0.01,0.01
8,1679.4091,3002.895960,,0.00,0.00


In [21]:
df_mean(df, fieldname)

Mean - Household Size
2014: 2.41849950512
2015: 2.44021059898


In [22]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Number of Children **

In [23]:
fieldname = 'Number of Children'
df = df_weighted(hh14, hh15, 'numchildren', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
numchildren,,,,,
0,999488.5356,1044083.290218,,0.68,0.68
1,207635.1024,195398.622727,,0.14,0.13
2,172140.3289,181851.733369,,0.12,0.12
3,63757.4392,65779.936522,,0.04,0.04
4,15645.8308,14303.676166,,0.01,0.01
5,2947.7428,24660.902942,,0.00,0.02
6,492.0145,NaN,,0.00,NaN


In [24]:
df_mean(df, fieldname)

Mean - Number of Children
2014: 0.563202423124
2015: 0.613966664205


In [25]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Number of Workers **

In [26]:
fieldname = 'Number of Workers'
df = df_weighted(hh14, hh15, 'numworkers', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
numworkers,,,,,
0,286020.4990,306814.447042,,0.20,0.20
1,594265.6443,618126.995490,,0.41,0.41
2,478981.2097,496173.985152,,0.33,0.33
3,81951.1563,80855.089218,,0.06,0.05
4,17946.5580,20908.122875,,0.01,0.01
5,1752.9723,3199.522169,,0.00,0.00
7,1188.9546,NaN,,0.00,NaN


In [27]:
df_mean(df, fieldname)

Mean - Number of Workers
2014: 1.29057265698
2015: 1.2795349442


In [28]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Income **

In [73]:
fieldname = 'Household Income'
df = df_weighted(hh14, hh15, 'hh_income_detailed', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
hh_income_detailed,,,,,
100k-150k,203805.9528,217376.020723,,0.14,0.14
10k-25k,158049.5916,146979.342115,,0.11,0.10
150k-200k,77315.1692,88526.171024,,0.05,0.06
200k-250k,36224.2372,36158.807396,,0.02,0.02
25k-35k,107948.6734,119552.282955,,0.07,0.08
35k-50k,169390.2994,175272.532789,,0.12,0.11
50k-75k,249011.1671,259117.002374,,0.17,0.17
75k-100k,179193.5086,172813.026843,,0.12,0.11
<10k,71032.5988,81168.998883,,0.05,0.05


In [88]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** County **

In [95]:
fieldname = 'County'
df = df_weighted(hh14, hh15, 'h_county_name', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
h_county_name,,,,,
King,793419.8858,808948.971527,,0.54,0.53
Kitsap,97665.0010,96341.278612,,0.07,0.06
Pierce,302475.1091,353511.224986,,0.21,0.23
Snohomish,268546.9983,267276.686819,,0.18,0.18


In [96]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** District **

In [98]:
fieldname = 'District'
df = df_weighted(hh14, hh15, 'h_district_name', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
h_district_name,,,,,
East Side,244912.5556,240095.695141,,0.17,0.16
Everett-Lynwood-Edmonds,111875.5679,110759.474921,,0.08,0.07
Kitsap,97665.0010,96341.278612,,0.07,0.06
North Seattle-Shoreline,144468.7298,142241.230849,,0.10,0.09
Renton-FedWay-Kent,201948.5012,215864.885587,,0.14,0.14
Seattle CBD,99845.8544,109335.480604,,0.07,0.07
South Pierce,202939.3474,159403.196203,,0.14,0.10
Suburban Snohomish,156671.4304,156517.211898,,0.11,0.10
Tacoma,98815.8924,194108.028783,,0.07,0.13


In [99]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Regional Growth Center **

In [255]:
fieldname = 'Regional Growth Center'
df = df_weighted(hh14, hh15, 'h_rgc_name', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
h_rgc_name,,,,,
Auburn,631.3311,2278.203430,,0.00,0.00
Bellevue,5841.7628,4584.332044,,0.00,0.00
Bothell Canyon Park,195.1288,NaN,,0.00,NaN
Bremerton,2161.4236,1092.103417,,0.00,0.00
Burien,375.4329,1785.593971,,0.00,0.00
Everett,2028.3215,3810.525439,,0.00,0.00
Kent,189.2049,NaN,,0.00,NaN
Kirkland Totem Lake,1175.2607,NaN,,0.00,NaN
Lakewood,2504.1317,2090.260904,,0.00,0.00


In [256]:
df.drop(df.index[df.index.get_loc('Outside RGC')],inplace=True)    # remove the no-growth-center row
df = df[[df1.year,df2.year]]
display_charts(df, kind='bar', title=fieldname)

** Seattle Urban Village **

In [274]:
fieldname = 'Seattle Urban Village'
df = df_weighted(hh14, hh15, 'h_uv_name', expwt14, expwt15)
numpy.

,2014,2015,,2014 Shares,2015 Shares
h_uv_name,,,,,
12th Avenue,1805.1624,441.333116,,0.00,0.00
23rd & Union-Jackson,4330.7518,10366.409348,,0.00,0.01
Admiral,872.5395,57.955363,,0.00,0.00
Aurora-Licton Springs,2500.7465,1211.734022,,0.00,0.00
Ballard,6621.9679,4782.606792,,0.00,0.00
Ballard-Interbay-Northend,700.6601,865.473673,,0.00,0.00
Belltown,5780.0962,2173.817681,,0.00,0.00
Bitter Lake Village,1910.5767,6636.960381,,0.00,0.00
Capitol Hill,17462.5989,9054.567828,,0.01,0.01


In [270]:
df = df[[df1.year,df2.year]]
display_charts(df, kind='bar', title=fieldname)

** Residence Type **

In [263]:
fieldname = 'Residence Type'
df = df_weighted(hh14, hh15, 'res_type', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_type,,,,,
Building with 3 or fewer apartments/condos,23909.5765,33315.023150,,0.02,0.02
Building with 4 or more apartments/condos,335936.5727,368190.805523,,0.23,0.24
Dorm or institutional housing,4637.0874,728.272526,,0.00,0.00
Mobile home/trailer,28065.1432,33009.219009,,0.02,0.02
Multi-family house (3 or fewer apartments),36328.3756,37046.750829,,0.02,0.02
"Other (including boat, RV, van, etc.)",3547.6595,1620.817353,,0.00,0.00
Single-family house (detached house),958122.6942,977258.127442,,0.66,0.64
Townhouse (attached house),71559.8851,74909.146112,,0.05,0.05


In [264]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

---

## Factors Influencing Residence Location

5 being very important,
1 very unimportant

**Importance of Walkability**

In [110]:
fieldname = 'Importance of Walkability'
df = df_weighted(hh14, hh15, 'res_factors_walk', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_factors_walk,,,,,
1,118989.1150,127199.397475,,0.08,0.08
2,131229.4275,130664.520395,,0.09,0.09
3,219378.2983,235978.268460,,0.15,0.15
4,484939.2250,487991.445058,,0.33,0.32
5,507570.9284,544244.530557,,0.35,0.36


In [111]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

**Importance of Walkability**

In [112]:
fieldname = 'Importance of Walkability'
df = df_weighted(hh14, hh15, 'res_factors_afford', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_factors_afford,,,,,
1,76861.3601,103665.910627,,0.05,0.07
2,44566.6699,58372.400541,,0.03,0.04
3,87241.2894,99716.780940,,0.06,0.07
4,378446.3723,405414.136999,,0.26,0.27
5,874991.3025,858908.932839,,0.60,0.56


In [113]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Importance of Transit Access**

In [114]:
fieldname = 'Importance of Transit Access'
df = df_weighted(hh14, hh15, 'res_factors_transit', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_factors_transit,,,,,
1,313443.7452,315789.130455,,0.21,0.21
2,199833.0247,178659.838315,,0.14,0.12
3,322417.9155,348031.165870,,0.22,0.23
4,307834.3613,393215.695874,,0.21,0.26
5,318577.9475,290382.331431,,0.22,0.19


In [115]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Household Changes **

In [123]:
fieldname = 'Household Changes'
df = df_weighted(hh14, hh15, 'res_factors_hhchange', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_factors_hhchange,,,,,
1,379082.1835,380045.107010,,0.26,0.25
2,65538.5241,92598.211950,,0.04,0.06
3,624031.0963,653587.919465,,0.43,0.43
4,196927.2705,213956.277631,,0.13,0.14
5,196527.9198,185890.645888,,0.13,0.12


In [124]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** School **

In [125]:
fieldname = 'School Access'
df = df_weighted(hh14, hh15, 'res_factors_school', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_factors_school,,,,,
1,352308.4163,418600.868778,,0.24,0.27
2,89494.4554,78149.846461,,0.06,0.05
3,397329.5240,367834.573804,,0.27,0.24
4,236242.1975,251793.968083,,0.16,0.16
5,386732.4010,409698.904818,,0.26,0.27


In [126]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Space **

In [128]:
fieldname = 'Space'
df = df_weighted(hh14, hh15, 'res_factors_space', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_factors_space,,,,,
1,132532.0786,195953.814804,,0.09,0.13
2,144446.1853,137478.251313,,0.10,0.09
3,287316.9586,276880.381796,,0.20,0.18
4,498168.8827,508284.748246,,0.34,0.33
5,399642.8890,407480.965787,,0.27,0.27


In [129]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Proximity to Family **

In [131]:
fieldname = 'Family'
df = df_weighted(hh14, hh15, 'res_factors_closefam', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_factors_closefam,,,,,
1,178709.9727,185832.310835,,0.12,0.12
2,159311.5886,136569.908335,,0.11,0.09
3,333036.5522,405792.586359,,0.23,0.27
4,459170.6686,503396.916171,,0.31,0.33
5,331878.2121,294486.440245,,0.23,0.19


In [132]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** Near a highway **

In [135]:
fieldname = 'Near highway'
df = df_weighted(hh14, hh15, 'res_factors_hwy', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_factors_hwy,,,,,
1,200313.7009,215245.742067,,0.14,0.14
2,212186.0887,212896.266309,,0.15,0.14
3,328156.2379,355047.210558,,0.22,0.23
4,529718.1006,553603.953562,,0.36,0.36
5,191732.8661,189284.989449,,0.13,0.12


In [136]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

** 30-minute commute **

In [138]:
fieldname = 'Achieve 30-minute commute'
df = df_weighted(hh14, hh15, 'res_factors_30min', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
res_factors_30min,,,,,
1,148848.1442,177932.979837,,0.10,0.12
2,95988.5494,83152.398513,,0.07,0.05
3,255617.1194,284121.461757,,0.17,0.19
4,354053.1236,381854.127195,,0.24,0.25
5,607600.0576,599017.194642,,0.42,0.39


In [141]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
display_charts(df, kind='bar', title=fieldname)

## Previous residence type

In [150]:
fieldname = 'Previous residence type'
df = df_weighted(hh14, hh15, 'prev_res_type', expwt14, expwt15)
df

,2014,2015,,2014 Shares,2015 Shares
prev_res_type,,,,,
Building with 3 or fewer apartments/condos,14712.1476,13134.933487,,0.01,0.01
Building with 4 or more apartments/condos,212537.6844,246219.474168,,0.15,0.16
Dorm or institutional housing,7076.8031,3719.010971,,0.00,0.00
Mobile home/trailer,7321.0032,1278.999969,,0.01,0.00
Multi-family house (3 or fewer apartments),30639.4444,27356.751460,,0.02,0.02
"Other (including boat, RV, van, etc.)",9280.7973,2797.057362,,0.01,0.00
Single-family house (detached house),256785.4291,294045.178178,,0.18,0.19
Townhouse (attached house),41684.8308,63073.244045,,0.03,0.04
no response,882068.8543,874453.512306,,0.60,0.57


In [157]:
df = df[[df1.year + ' Shares',df2.year + ' Shares']]
df.drop(df.index[-1],inplace=True)    # remove the no-response fields
display_charts(df, kind='bar', title=fieldname)